In [31]:
import pandas as pd
sheet_url='https://docs.google.com/spreadsheets/d/1MvIxVRBs4u6UFfvnu2khnU_gtrcoLUch5Tci-hcdkAk/edit#gid=635530870'
csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')

df = pd.read_csv(csv_export_url)
df

,Sentence,Label
0,NaN,1
1,""" or pg_sleep ( __TIME__ ) --",1
2,create user name identified by pass123 tempora...,1
3,%29,1
4,AND 1 = utl_inaddr.get_host_address ( ( SE...,1
...,...,...
37939,"org/?option = com_k2 <a href = ""http://corfopym",0
37940,com/?option = com_k2 <act> <![CDATA[procMemb...,0
37941,picsearch,0
37942,com/is?-WZx-uhyLezKNiYLvAbKL3W4oh5F749nr2KUmFF...,0


In [32]:
sentence_col = df.drop(['Label'],axis='columns')
Y = df['Label']
sentence_col

,Sentence
0,NaN
1,""" or pg_sleep ( __TIME__ ) --"
2,create user name identified by pass123 tempora...
3,%29
4,AND 1 = utl_inaddr.get_host_address ( ( SE...
...,...
37939,"org/?option = com_k2 <a href = ""http://corfopym"
37940,com/?option = com_k2 <act> <![CDATA[procMemb...
37941,picsearch
37942,com/is?-WZx-uhyLezKNiYLvAbKL3W4oh5F749nr2KUmFF...


# **Feature Extraction**

In [33]:
#  Detect select/update/insert/delete keyword at the initial part of the query

def isFetchCommandType(query):

    intial_keyword = query.split(' ')[0].lower()
    if( intial_keyword == 'select' ):
        return 1
    else:
        return 0

def isUpdateCommandType(query):
    intial_keyword = query.split(' ')[0].lower()
    if( intial_keyword == 'update' ):
        return 1
    else:
        return 0
      
def isInsertCommandType(query):
    intial_keyword = query.split(' ')[0].lower()
    if( intial_keyword == 'insert' ):
        return 1
    else:
        return 0

def isDeleteCommandType(query):
    intial_keyword = query.split(' ')[0].lower()
    if( intial_keyword == 'delete' ):
        return 1
    else:
        return 0


print(isFetchCommandType("INSERT into(223,'sajid','2343113')") )
print(isUpdateCommandType("DELETE into(223,'brishty','2343113')") )
print(isInsertCommandType("DELETE into(223,'sajid','2343113')") )
print(isDeleteCommandType("DELETE into(223,'brishty','2343113')") )

0
0
0
1


In [34]:
# Detect 'order by' keyword in the query

def detectSortClause(query):
    keyword='order by'
    if(keyword in query.lower()):
        return 1
    else:
        return 0
    
print(detectSortClause("-5021'+ ( select yadq where 4285 = 4285 Order By 1#"))

1


In [35]:
  # Detect group by keyword
  def detectGroupClause(query):
      keyword= 'group by'  
      if(keyword in query.lower()):
          return 1
      else:
          return 0

print(detectGroupClause("-5021'+ ( select yadq where 4285 = 4285 Group By 1#"))

1


In [36]:
# Detect over keyword 

def detectWindowClause(query):
    keyword='over'
    if(keyword in query.lower()):
        return 1
    else:
        return 0

print(detectWindowClause("-5021'+ ( select yadq where 4285 = 4285 order by 1#"))

0


In [37]:
#hasRecursive: Indicator if SQL is specified WITH RECURSIVE clause
def detectWithRecursive(query):
    keyword='with recursive'
    if(keyword in query.lower()):
        return 1
    else:
        return 0
print(detectWithRecursive('With Recursive parent_of(name, parent) AS (SELECT name, mom FROM family UNION SELECT name, dad FROM family);'))


1


In [38]:
#hasDistinctOn: Indicator if distinct clause is from DISTINCT ON
def detecthasDistinctOn(query):
    keyword= 'distinct on'
    if(keyword in query.lower()):
        return 1
    else:
        return 0

print(detecthasDistinctOn('SELECT * FROM (SELECT DISTINCT ON(address_id) * FROM purchases WHERE product_id = 1 ORDER BY address_id, purchased_at DESC) t ORDER BY purchased_at DESC;'))
print(detecthasDistinctOn('SELECT * FROM First JOIN SELECT * FROM Second;')  )


1
0


In [39]:
#utilityStmt: Indicator if the SQL is DECLARE CURSOR or a non-optimizable statement

def utilityStmt(query):
    keyword1='declare'
    keyword2='cursor'
    if(keyword1 in query.lower() and keyword2 in query.lower()):
      return 1
    else:
      return 0

print(utilityStmt('EXEC SQL DECLARE C3 CURSOR WITH HOLD FOR SELECT * FROM EMPLOYEE FOR UPDATE OF WORKDEPT, PHONENO, JOB, EDLEVEL, SALARY;'))
print(utilityStmt('SELECT * FROM First JOIN SELECT * FROM Second;')  )


1
0


In [40]:
#jointree/quals/args*/boolop* : Qualification type BOOLEXPR, OPEXPR and boolean operation and, or, not
# (contains_logical_operator)
def hasBoolOperator(query):
      keyword1='and'
      keyword2='or'
      keyword3='not'
      if(keyword1 in query.lower() or keyword2 in query.lower() or keyword3 in query.lower()):
        return 1
      else:
        return 0
print(hasBoolOperator('SELECT column-names FROM table-name WHERE condition1 AND condition2;'))
print(hasBoolOperator('UPDATE table-name SET column-name = value WHERE condition1 OR condition2;'))
print(hasBoolOperator('DELETE table-name WHERE NOT condition;'))

1
1
1


**List of window functions**: https://www.sqlshack.com/use-window-functions-sql-server/

In [41]:
# detect window function in the query RANK(), DENSE_RANK(), ROW_NUMBER(), NTILE(),SUM(), MAX(), MIN(), AVG(). COUNT(),LAG(), LEAD(), FIRST_VALUE(), LAST_VALUE(),
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

def hasWindowFunc(query):
    window_func_list=['sum','avg','max','min','count','rank','dense_rank','row_number','nitle','lag','lead','first_value','last_value']
    tokenized_list = word_tokenize(query.lower())
   
    #print(tokenized_list)
    for window_func in window_func_list:
        #print(window_func)
        for i in range(len(tokenized_list)):
            if(window_func == tokenized_list[i]):
                if( tokenized_list[i+1] == '('):
                    return 1
                else:
                    return 0
    return 0



hasWindowFunc("SELECT city, SUM(order_amount) total_order_amount FROM sum GROUP BY city")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


1

In [42]:
# hasAggs: Indicator if SQL has aggregates (sum(),avg(),max(),min(),count())
def hasAggs(query):
    Agg_func_list=['sum','avg','max','min','count']
    tokenized_list = word_tokenize(query.lower())
    
    #print(tokenized_list)
    for Agg_func in Agg_func_list:
        #print(window_func)
        for i in range(len(tokenized_list)):
            if(Agg_func == tokenized_list[i]):
                if( tokenized_list[i+1] == '('):
                    return 1
                else:
                    return 0
    return 0

    
print(hasAggs("-5021'+ avg( select yadq where 4285 = 4285 order by 1#"))
print(utilityStmt('SELECT * FROM First JOIN SELECT * FROM Second;')  )
print(hasAggs("-5021'+ max( select yadq where 4285 = 4285 order by 1#"))
print(hasAggs("-5021'+ min( select yadq where 4285 = 4285 order by 1#"))
print(hasAggs("-5021'+ count( select yadq where 4285 = 4285 order by 1#"))

1
0
1
1
1


In [43]:
#detect set operations like union,union all, except, intersect

def hasSetOperation(query):
    set_list=['union','union all','except','intersect']
    tokenized_list = word_tokenize(query.lower())
   
    #print(tokenized_list)
    for set_op in set_list:
        #print(window_func)
        for i in range(len(tokenized_list)):
            if(set_op in tokenized_list):
                return 1
            
    return 0

print(hasSetOperation('SELECT * FROM First JOIN SELECT * FROM Second;'))

0


In [44]:
#cteList---> getNumberOfWithClause() 
# detect number of with clasue 

def getNumberOfWithClause(query) :
    keyword='with'
    
    tokenized_list = word_tokenize(query.lower())
    print(tokenized_list)
    cnt=0
    for word in tokenized_list:
        if(keyword == word ):
            cnt +=1
    
    return cnt    




print(getNumberOfWithClause( ' WITH totalSalary(Airline, total) as (SELECT Airline, sum(Salary) FROM Pilot GROUP BY Airline),airlineAverage(avgSalary) as (SELECT avg(Salary) FROM Pilot ) WITH SELECT Airline FROM totalSalary, airlineAverage WHERE totalSalary.total > airlineAverage.avgSalary;'))

['with', 'totalsalary', '(', 'airline', ',', 'total', ')', 'as', '(', 'select', 'airline', ',', 'sum', '(', 'salary', ')', 'from', 'pilot', 'group', 'by', 'airline', ')', ',', 'airlineaverage', '(', 'avgsalary', ')', 'as', '(', 'select', 'avg', '(', 'salary', ')', 'from', 'pilot', ')', 'with', 'select', 'airline', 'from', 'totalsalary', ',', 'airlineaverage', 'where', 'totalsalary.total', '>', 'airlineaverage.avgsalary', ';']
2


In [49]:
# detect sub query in the query

def detectSubQuery(query):
    commands = ['select','update','insert','delete']
    tokenized_list = word_tokenize(query.lower())
    cnt=0
    for command in commands:
        if(command in tokenized_list):
            cnt+=1
    if(cnt>1):
        return 1
    else:
        return 0


print(detectSubQuery('UPDATE CUSTOMERS SET SALARY = SALARY * 0.25 WHERE AGE IN ( SELECT AGE FROM CUSTOMERS_BKP WHERE AGE >= 27 );'))

print(detectSubQuery('INSERT INTO CUSTOMERS_BKP SELECT * FROM CUSTOMERS WHERE ID IN (SELECT ID FROM CUSTOMERS) ;'))

1
1
